In [ ]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Hand model
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize a video capture object (you can also use an image)
cap = cv2.VideoCapture(0)  # 0 for the default camera

# Define gesture recognition functions
def recognize_gesture(hand_landmarks, landmark_indices, threshold=0.0):
    # Check if the specified landmarks have similar y-coordinates
    y_coordinates = [hand_landmarks.landmark[i].y for i in landmark_indices]
    min_y = min(y_coordinates)
    max_y = max(y_coordinates)
    
    if max_y - min_y < threshold:
        return True
    else:
        return False

def recognize_thumbs_up(hand_landmarks):
    landmark_indices = [4, 8, 12, 16, 20]  # Indices of thumb and fingers' tips
    if recognize_gesture(hand_landmarks, landmark_indices, threshold=0.05):
        return "Thumbs Up"
    return "No Gesture"

def recognize_thumbs_down(hand_landmarks):
    landmark_indices = [4, 8, 12, 16, 20]  # Indices of thumb and fingers' tips
    if recognize_gesture(hand_landmarks, landmark_indices, threshold=0.05):
        return "Thumbs Down"
    return "No Gesture"

def recognize_peace_sign(hand_landmarks):
    landmark_indices = [8, 12, 16, 20]  # Indices of fingers' tips (excluding thumb)
    if recognize_gesture(hand_landmarks, landmark_indices, threshold=0.1):
        return "Peace Sign"
    return "No Gesture"

def recognize_fist(hand_landmarks):
    landmark_indices = [0, 4, 8, 12, 16]  # Indices of palm and fingers' bases
    if recognize_gesture(hand_landmarks, landmark_indices, threshold=0.05):
        return "Fist"
    return "No Gesture"

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue

    # Convert the BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform hand pose estimation
    hand_results = hands.process(rgb_frame)

    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            # Draw landmarks for the hand (changed the color to blue)
            for landmark in hand_landmarks.landmark:
                x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 5, (255, 0, 0), -1)  # Blue color

            # Recognize the gestures
            gesture_thumbs_up = recognize_thumbs_up(hand_landmarks)
            gesture_thumbs_down = recognize_thumbs_down(hand_landmarks)
            gesture_peace_sign = recognize_peace_sign(hand_landmarks)
            gesture_fist = recognize_fist(hand_landmarks)

            # Display the recognized gestures
            cv2.putText(frame, gesture_thumbs_up, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(frame, gesture_thumbs_down, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(frame, gesture_peace_sign, (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(frame, gesture_fist, (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow('Gesture Detection', frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Press 'Esc' to exit
        break

cap.release()
cv2.destroyAllWindows()